In [15]:
import numpy as np

def alpha(p):
    """Returns the alpha coefficient for the DCT formula"""
    return 1.0 / np.sqrt(2.0) if p == 0 else 1.0

def dct_2d_channel_block(image_block):
    """Performs 2D DCT on a single block (e.g., 8x8) of the image"""
    w, h = image_block.shape
    dct_block = np.zeros((w, h))

    for u in range(w):
        for v in range(h):
            sum_val = 0
            for x in range(w):
                for y in range(h):
                    sum_val += image_block[x, y] * np.cos(((2 * x + 1) * u * np.pi) / (2 * w)) * np.cos(((2 * y + 1) * v * np.pi) / (2 * h))
            dct_block[u, v] = 0.25 * alpha(u) * alpha(v) * sum_val

    return dct_block

def idct_2d_channel_block(dct_block):
    """Performs 2D IDCT on a single block (e.g., 8x8) of the image"""
    w, h = dct_block.shape
    idct_block = np.zeros((w, h))

    for x in range(w):
        for y in range(h):
            sum_val = 0
            for u in range(w):
                for v in range(h):
                    sum_val += alpha(u) * alpha(v) * dct_block[u, v] * np.cos(((2 * x + 1) * u * np.pi) / (2 * w)) * np.cos(((2 * y + 1) * v * np.pi) / (2 * h))
            idct_block[x, y] = 0.25 * sum_val

    return idct_block

def dct_2d(image, block_size=8):
    """Performs 2D DCT on an image with multiple channels using sliding window"""
    w, h, c = image.shape
    dct_image = np.zeros((w, h, c))

    # Process the image block by block
    for ch in range(c):
        for i in range(0, w, block_size):
            for j in range(0, h, block_size):
                # Get the block
                block = image[i:i+block_size, j:j+block_size, ch]
                # Apply DCT on the block
                dct_block = dct_2d_channel_block(block)
                # Store the result in the corresponding position
                dct_image[i:i+block_size, j:j+block_size, ch] = dct_block

    return dct_image

def idct_2d(dct_image, block_size=8):
    """Performs 2D IDCT on an image with multiple channels using sliding window"""
    w, h, c = dct_image.shape
    idct_image = np.zeros((w, h, c))

    # Process the image block by block
    for ch in range(c):
        for i in range(0, w, block_size):
            for j in range(0, h, block_size):
                # Get the block
                dct_block = dct_image[i:i+block_size, j:j+block_size, ch]
                # Apply IDCT on the block
                idct_block = idct_2d_channel_block(dct_block)
                # Store the result in the corresponding position
                idct_image[i:i+block_size, j:j+block_size, ch] = idct_block

    return idct_image

# Example usage:

# Create a random 32x32 RGB image
image = np.random.rand(224, 224, 3)

# Apply 2D-DCT with block size of 8
dct_result = dct_2d(image, block_size=8)

# Apply 2D-IDCT to recover the original image
idct_result = idct_2d(dct_result, block_size=8)

# Print results for testing
print("Original Image:\n", image)
print("DCT Result:\n", dct_result)
print("IDCT Reconstructed Image:\n", idct_result)

# Check if the original and reconstructed images are close
print("Reconstruction Error:\n", np.abs(image - idct_result).mean())

Original Image:
 [[[0.97510306 0.46428133 0.68504411]
  [0.94119336 0.12565063 0.81680799]
  [0.94423859 0.12724204 0.77420324]
  ...
  [0.98791387 0.06264774 0.78884065]
  [0.78531145 0.47597057 0.1388484 ]
  [0.28035375 0.71025921 0.98880802]]

 [[0.44177815 0.64843623 0.62658591]
  [0.54345009 0.66884982 0.4854427 ]
  [0.54068426 0.3352705  0.30481425]
  ...
  [0.28559641 0.96587848 0.19846824]
  [0.11230081 0.2765603  0.76871627]
  [0.98032281 0.45492745 0.86171242]]

 [[0.5061728  0.95916076 0.18146066]
  [0.2252624  0.00593918 0.1815194 ]
  [0.11579679 0.05441222 0.7447968 ]
  ...
  [0.27144028 0.81325218 0.42227344]
  [0.65161275 0.47268545 0.98070014]
  [0.79487614 0.30598026 0.34815139]]

 ...

 [[0.22114298 0.02110512 0.48017457]
  [0.73279973 0.92008616 0.97254948]
  [0.74452359 0.85919306 0.42819596]
  ...
  [0.77733647 0.26237266 0.03995948]
  [0.36469584 0.71300694 0.07398485]
  [0.18422134 0.35887846 0.05292121]]

 [[0.33308094 0.08946278 0.18788508]
  [0.74647098 0.8692

In [12]:
x_0 = Image.open('../resource/example/000001.jpg')
x_0 = np.array(x_0)
x_0 = np.resize(x_0, (8, 8, 3))
x_0[0:5, 0:5, 0]

array([[253, 253, 253, 253, 253],
       [253, 253, 253, 253, 253],
       [254, 254, 254, 254, 254],
       [254, 253, 253, 253, 253],
       [251, 251, 252, 253, 253]], dtype=uint8)

In [13]:
x_dct = np.zeros_like(x_0)
for i in range(x_0.shape[2]):
    x_dct[:, :, i] = dct_2d_channel(x_0[:, :, i])
x_dct[0:5, 0:5, 0]

array([[233, 255,   0,   0,   0],
       [  0,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [253,   0,   0,   0,   0],
       [  0, 255,   0,   0,   0]], dtype=uint8)

In [14]:
x_idct = np.zeros_like(x_0)
for i in range(x_0.shape[2]):
    x_idct[:, :, i] = idct_2d_channel(x_dct[:, :, i])
x_idct[0:5, 0:5, 0].astype(np.uint8)

array([[175, 155, 120,  81,  46],
       [219, 238,   8,  27,  27],
       [ 71,  43,   2, 231, 231],
       [248,  11,  34,  35,   0],
       [243, 196, 124,  58,  23]], dtype=uint8)